In [84]:
import sys
import traceback
from memsql.common import database

HOST = 'svc-9b7ada2b-bfbb-48de-b41f-ce66e2827760-dml.azr-pune-1.svc.singlestore.com'
PORT = 3306
USER = 'admin'
PASSWORD = 'Devpost#123'
DATABASE = 'cropdb'

conn = database.connect(host=HOST, port=PORT, user=USER, password=PASSWORD, database=DATABASE)

def read_all(conn):
    sql = "SELECT * FROM croprecommend"
    rows = conn.query(sql)
    return rows

try:
    conn.ping()
    rows = read_all(conn)
#     for row in rows:
#       print(row, sep ='\t')
#     print(rows)

except Exception as e:
    print("Error")
    print(e)
    traceback.print_exc(file =sys.stdout)

finally:
    conn.close()

In [85]:
import pandas as pd
df = pd.DataFrame([test.values() for test in rows], columns=['N','P','K','temperature','humidity','ph','rainfall','label'])
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,67,59,41,21.9477,80.9738,6.01263,213.356,rice\r
1,63,44,41,24.1730,83.7288,5.58337,257.034,rice\r
2,62,42,36,22.7813,82.0672,6.43001,248.718,rice\r
3,90,44,38,23.8351,83.8839,7.47313,241.201,rice\r
4,76,44,17,20.4168,62.5542,5.85544,65.278,maize\r


In [86]:
df = df.drop(['P','K'], axis=1)

In [87]:
df.shape

(2201, 6)

In [88]:
df.head()

,N,temperature,humidity,ph,rainfall,label
0,67,21.9477,80.9738,6.01263,213.356,rice\r
1,63,24.1730,83.7288,5.58337,257.034,rice\r
2,62,22.7813,82.0672,6.43001,248.718,rice\r
3,90,23.8351,83.8839,7.47313,241.201,rice\r
4,76,20.4168,62.5542,5.85544,65.278,maize\r


In [89]:
df.isnull().sum()

N              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64

In [90]:
corrmat = df.corr()

In [91]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [92]:
corrmat

,N,temperature,humidity,ph,rainfall
N,1.000000,0.029469,0.192155,0.100263,0.060118
temperature,0.029469,1.000000,0.210982,0.001396,-0.025586
humidity,0.192155,0.210982,1.000000,0.003651,0.096866
ph,0.100263,0.001396,0.003651,1.000000,-0.100255
rainfall,0.060118,-0.025586,0.096866,-0.100255,1.000000


In [93]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [94]:
df.head()

,N,temperature,humidity,ph,rainfall,label
0,67,21.9477,80.9738,6.01263,213.356,21
1,63,24.1730,83.7288,5.58337,257.034,21
2,62,22.7813,82.0672,6.43001,248.718,21
3,90,23.8351,83.8839,7.47313,241.201,21
4,76,20.4168,62.5542,5.85544,65.278,12


In [95]:
x = df.drop(['label'],axis=1)
y = df['label']

In [96]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size = 0.2, random_state=2)

In [97]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [98]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12) ]
max_features = ['auto','sqrt']
max_depth = [int(x) for x in np.linspace(5,30, num=6)]
min_samples_split = [2,5,10,15,100]
min_samples_leaf = [1,2,5,10]

In [99]:
from sklearn.model_selection import RandomizedSearchCV

In [100]:
random_grid = { 'n_estimators': n_estimators,
              'max_features':max_features,
              'max_depth':max_depth,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf }
print(random_grid)

{'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200], 'max_features': ['auto', 'sqrt'], 'max_depth': [5, 10, 15, 20, 25, 30], 'min_samples_split': [2, 5, 10, 15, 100], 'min_samples_leaf': [1, 2, 5, 10]}


In [101]:
 rf_random = RandomizedSearchCV(estimator=rf, param_distributions = random_grid, scoring = 'neg_mean_squared_error',
                                n_iter = 10, cv = 5, verbose = 2, random_state=42, n_jobs=1)

In [102]:
ytrain

868      3
1940     9
784      6
372      3
696      1
        ..
1071     7
433      3
674      9
1099    12
1608     5
Name: label, Length: 1760, dtype: int32

In [ ]:
rf_random.fit(xtrain,ytrain)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   5.8s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   5.5s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   5.7s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   5.1s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   4.2s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=   5.6s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=   7.4s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimator

In [ ]:
rf_random.best_params_

In [ ]:
y.value_counts()

In [ ]:
pred = rf_random.predict(xtest)

In [ ]:
df=pd.DataFrame({'Actual':ytest, 'Predicted':pred})
df

In [ ]:
from sklearn import metrics
print(metrics.mean_absolute_error(ytest, pred))
print(metrics.mean_squared_error(ytest, pred))
print(np.sqrt(metrics.mean_squared_error(ytest, pred)))


In [ ]:
import pickle
file = open('finalizedresult.pkl','wb')
pickle.dump(rf_random,file)